In [3]:
import warnings
warnings.filterwarnings('ignore')
#Read the .h5 file
import h5py
#Build spreadSheets
import pandas as pd
#mathematical operations and array functions
import numpy as np
#Building and Using radar objecta
import pyart
#Plotting graphs
import matplotlib.pyplot as plt
#Getting computer's info
import os
#Same
import sys
#Getting the map for radar plottings
import cartopy.crs as ccrs
#Warning do not edit the utilities file 
from utilities import *
import datetime

In [4]:
h5 = h5py.File('../data/all_data/202202130255_polar_pl_radar08b0_augzdr_lp_canonical', 'r+')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../data/all_data/202202130255_polar_pl_radar08b0_augzdr_lp_canonical', errno = 2, error message = 'No such file or directory', flags = 1, o_flags = 2)

In [5]:
def create_single_radar_file(h5filename, originalfilename, location):
    
    if h5filename["what"].attrs["object"] == b'scan': #In this case it isn't, it's already SCAN
        h5filename["what"].attrs.__setitem__("object", "SCAN")
        
    radar = pyart.aux_io.read_odim_h5(location + "/" + originalfilename)
    return radar
    dfile.close()

In [6]:
create_single_radar_file(h5, "202201070000_polar_pl_radar08b3_augzdr_sp_canonical.h5", "../data/all_files")

NameError: name 'h5' is not defined

In [7]:
def select_files_angle(route, angle):
    fileDir = os.listdir(route)
    #gives a list of the files present at route
    radarList = []
    #defining an empty radar list to populate
    for file in fileDir:
        #This for loop cycles through all the files in fileDir and makes a temporary variable dfile assigned to the current file in question in the cycle
        dfile =  h5py.File(route + "/" + file , 'r')
        try:
            #the temporary dfile is then 'tried'.
            if dfile["what"].attrs["object"] == b'scan':
                dfile["what"].attrs.__setitem__("object", "SCAN")
            #I believe this for loop changes what>object to SCAN from b'scan' - if ever what>object is in fact b'scan', I don't know
            if dfile["dataset1"]["where"].attrs["elangle"] == angle:
                radar = pyart.aux_io.read_odim_h5(route +"/"+file)
            #if the elangle under dataset1 is less than/equal to the chosen angle create the radar file, and add it to radarList
                radarList.append(radar)
            dfile.close()
        except:
            dfile.close()
    return radarList

In [8]:
select_files_angle("../data/test_files", angle = 0.5)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/test_files'

In [ ]:
def get_angle(fileRoute):
    
    try:
        dfile =  h5py.File(fileRoute, "r")
        a = dfile["dataset1"]["where"].attrs["elangle"]
        return a
        dfile.close()
    except:
        return np.nan

In [ ]:
def build_readings_df(route):
    df = pd.DataFrame(columns = ["File Name", "Date", "Hour", "Angle", "Pulse"])
    df["File Name"] = [route + "/"+  file for file in os.listdir(route)]
    df["Date"] = df["File Name"].apply(lambda x : int(x.split("/")[-1].split("_")[0][:8]))
    df["Hour"] = df["File Name"].apply(lambda x : int(x.split("/")[-1].split("_")[0][8:]))
    df["Pulse"] = df["File Name"].apply(lambda x : x.split("_")[6])
    df["Angle"] = df["File Name"].apply(lambda x : get_angle(x))
    return df

In [ ]:
build_readings_df("../data/all_data")

In [ ]:
df = build_readings_df("../data/all_data")
df.to_csv("radar_df.csv")

In [ ]:
#To select a subset of the data table we use this:
#You can edit the values of the columns to change the selected data
selected_data = df.loc[(df["Pulse"]== "lp")&  (df["Angle"]== 0.5)] # & ((df["Date"]== 20220204) | (df["Date"]== 20220204))]

In [ ]:
selected_data

In [ ]:
def get_radar_list(df, date=None, hour=None, angle=None, pulse=None):
    
    if date is not None:
        df = df.loc[df['Date'] == date]
    if hour is not None:
        df = df.loc[df['Hour'] == hour]
    if angle is not None:
        df = df.loc[df['Angle'] == angle]
    if pulse is not None:
        df = df.loc[df['Pulse'] == pulse]
    listRadars = []
    for route in df["File Name"].values:
        h5 = h5py.File(route, 'r')
        originalfilename = route[19:]
        location = route[:18]
        listRadars.append(create_single_radar_file(h5, originalfilename, location))
    return listRadars

In [ ]:
def plot_graph(radar, vmax =None,vmin=None,gatefilter=None,name=None,save = False):
    
    display = pyart.graph.RadarMapDisplay(radar)
    projection = ccrs.LambertConformal(central_latitude=radar.latitude['data'][0],
                                       central_longitude=radar.longitude['data'][0])
    fig, #axs = plt.subplots(nrows=2, ncols=3,figsize=(30,15),
                                #subplot_kw={'projection': projection})
    display.plot_ppi_map("total_power_horizontal"[0],
                         resolution='10m',
                         #projection=projection,
                         #fig=fig, #ax = axs[0][0],
                         lat_0=radar.latitude['data'][0],
                         lon_0=radar.longitude['data'][0],
                         min_lon=radar.longitude['data'][0]+1.10, max_lon=radar.longitude['data'][0]-1.10, min_lat=radar.latitude['data'][0]+0.6, max_lat=radar.latitude['data'][0]-0.8,
                         gatefilter=gatefilter,
                         vmax = vmax,
                         vmin = vmin
                         )
    display.plot_ray("total_power_horizontal", 1)
    display.plot_point(radar.longitude['data'][0], radar.latitude['data'][0])
    display.plot_range_rings([35.,60.], ls='--') 


    if save:
        plt.savefig(name)
    plt.show()

In [ ]:
#This makes a list of radar objects for the selected data above
radarList = get_radar_list(selected_data, date="20220106")

In [ ]:
ListofAverages = []
ListofTimes = []

for i in range(0,len(radarList)-1,1):
    radar1= radarList[i]    
    name=""+ "-".join(radar1.time["units"].split(" ")[-1].split(":"))
    
    time ="" + "".join(radar1.time["units"].split("T")[-1].split("Z"))
    
    print(time)
    
    gatefilter_lp = pyart.correct.GateFilter(radar1)  
    gatefilter_lp.exclude_outside('normalized_coherent_power',0.2,0.8)
    gatefilter_lp.exclude_inside('cross_correlation_ratio', 0.7,1.2)
    
    rayrange = np.arange(start=90, stop=270, step=1)

    Total = 0
    counter = 0

    for ray in rayrange: 

        mask = gatefilter_lp.gate_excluded[ray]

        consecutive_true_count = 0
        for i, value in enumerate(mask):
            if value:
                consecutive_true_count += 1
                if consecutive_true_count == 21:
                    EndOfBackscatter = i-consecutive_true_count+1
                    #print(f"%s consecutive True values start at index {i-consecutive_true_count+1}" % consecutive_true_count)
                    break
            else:
                consecutive_true_count = 0

        #print(f"This Ray's Backscatter ends after Bin {EndOfBackscatter}")

        BackscatterRadius = EndOfBackscatter*0.6

        #print(f"each bin is 0.6km in length and therefore this index corresponds to a distance of %s" % BackscatterRadius + "m")
        #print(f"Backscatter Radius azimuthal angle {ray}\N{DEGREE SIGN} (clockwise from North) %s" % BackscatterRadius + "m")
        counter = counter + 1
        Total = Total + BackscatterRadius


    Average = Total/counter
    print("For Date/Time: " + name + f", the average Backscatter Radius was = %s" % Average + "m")
    ListofAverages.append(Average)
    ListofTimes.append(time)

In [ ]:
y_raw = ListofAverages
x = ListofTimes
#y_rolling_av = moving_average(y_raw, 10)

data = {'Backscatter Radius' : ListofAverages, 
        'time' : ListofTimes}

plt.figure(1)
plt.plot(data['time'] ,data['Backscatter Radius'])
plt.gcf().autofmt_xdate()
plt.title('Backscatter Radius against Time')
plt.show()